In [64]:
import re, string

class ReadInput:
    
    _input_vars = {
        'model_output__site_prefix': {'name':'site_prefix', 'type': 'string', 'default': ''},
        'model_output__case_prefix': {'name':'case_prefix', 'type': 'string', 'default': ''},
        'model_output__out_dir': {'name':'out_dir', 'type': 'string', 'default': '~/deltaRCM_Output/'},
        'model__total_timesteps': {'name':'n_steps', 'type': 'long', 'default': 200},
        'model_grid__length': {'name':'Length', 'type': 'float', 'default': 200.},
        'model_grid__width': {'name':'Width', 'type': 'float', 'default': 500.},
        'model_grid__cell_size': {'name':'dx', 'type': 'float', 'default': 10.},
        'land_surface__width': {'name':'L0_meters', 'type': 'float', 'default': 30.}, 
        'land_surface__slope': {'name':'S0', 'type': 'float', 'default': 0.00015},
        'model__max_iteration': {'name':'itermax', 'type': 'long', 'default': 3},
        'water__number_parcels': {'name':'Np_water', 'type': 'long', 'default': 200},
        'channel__flow_velocity': {'name':'u0', 'type': 'float', 'default': 1.},
        'channel__width': {'name':'N0_meters', 'type': 'float', 'default': 50.},
        'channel__flow_depth': {'name':'h0', 'type': 'float', 'default': 5.},
        'sea_water_surface__elevation': {'name':'H_SL', 'type': 'float', 'default': 0.},
        'sea_water_surface__rate_change_elevation': {'name':'SLR', 'type': 'float', 'default': 0.},
        'sediment__number_parcels': {'name':'Np_sed', 'type': 'long', 'default': 500},
        'sediment__bedload_fraction': {'name':'f_bedload', 'type': 'float', 'default': 0.25}, 
        'sediment__influx_concentration': {'name':'C0_percent', 'type': 'float', 'default': 0.1},                   
        'model_output__opt_eta_figs': {'name':'save_eta_figs', 'type': 'choice', 'default': False},
        'model_output__opt_stage_figs': {'name':'save_stage_figs', 'type': 'choice', 'default': False},
        'model_output__opt_depth_figs': {'name':'save_depth_figs', 'type': 'choice', 'default': False},
        'model_output__opt_eta_grids': {'name':'save_eta_grids', 'type': 'choice', 'default': False},
        'model_output__opt_stage_grids': {'name':'save_stage_grids', 'type': 'choice', 'default': False},
        'model_output__opt_depth_grids': {'name':'save_depth_grids', 'type': 'choice', 'default': False},
        'model_output__opt_time_interval': {'name':'save_dt', 'type': 'long', 'default': 10},
        'coeff__surface_smoothing': {'name': 'Csmooth', 'type': 'float', 'default': 0.9},
        'coeff__under_relaxation__water_surface': {'name': 'omega_sfc', 'type': 'float', 'default': 0.1},
        'coeff__under_relaxation__water_flow': {'name': 'omega_flow', 'type': 'float', 'default': 0.9},
        'coeff__iterations_smoothing_algorithm': {'name': 'Nsmooth', 'type': 'long', 'default': 5},
        'coeff__depth_dependence__water': {'name': 'theta_water', 'type': 'float', 'default': 1.0},
        'coeff__depth_dependence__sand': {'name': 'coeff_theta_sand', 'type': 'float', 'default': 2.0},
        'coeff__depth_dependence__mud': {'name': 'coeff_theta_mud', 'type': 'float', 'default': 1.0},
        'coeff__non_linear_exp_sed_flux_flow_velocity': {'name': 'beta', 'type': 'long', 'default': 3},
        'coeff__sedimentation_lag': {'name': 'sed_lag', 'type': 'float', 'default': 1.0},
        'coeff__velocity_deposition_mud': {'name': 'coeff_U_dep_mud', 'type': 'float', 'default': 0.3},
        'coeff__velocity_erosion_mud': {'name': 'coeff_U_ero_mud', 'type': 'float', 'default': 1.5},
        'coeff__velocity_erosion_sand': {'name': 'coeff_U_ero_sand', 'type': 'float', 'default': 1.05},
        'coeff__topographic_diffusion': {'name': 'alpha', 'type': 'float', 'default': 0.1}
        }


    def get_var_name(self, long_var_name): 
        return self._var_name_map[ long_var_name ]
    
    def import_file(self):
        verbose = True

        if verbose: print 'Reading input file...'

        input_file = 'deltaRCM.in'
        self.input_file_vars = dict()
        numvars = 0

        o = open(input_file, mode = 'r')

        for line in o:
            line = re.sub('\s$','',line)
            line = re.sub('\A[: :]*','',line)
            ln = re.split('\s*[\:\=]\s*', line)

            if len(ln)>1:

                ln[0] = string.lower(ln[0])

                if ln[0] in self._input_var_names:

                    numvars += 1

                    var_type = self._var_type_map[ln[0]]

                    ln[1] = re.sub('[: :]+$','',ln[1])

                    if var_type == 'string':
                        self.input_file_vars[str(ln[0])] = str(ln[1])
                    if var_type == 'float':
                        self.input_file_vars[str(ln[0])] = float(ln[1])
                    if var_type == 'long':
                        self.input_file_vars[str(ln[0])] = int(ln[1])
                    if var_type == 'choice':

                        ln[1] = string.lower(ln[1])

                        if ln[1] == 'yes' or ln[1] == 'true':
                            self.input_file_vars[str(ln[0])] = True
                        elif ln[1] == 'no' or ln[1] == 'false':
                            self.input_file_vars[str(ln[0])] = False
                        else:
                            print "Alert! The option for the 'choice' type variable " \
                                  "in the input file '" + str(ln[0]) + "' is unrecognized. " \
                                  "Please use only Yes/No or True/False as values.\n"

                else:
                    print "Alert! The input file contains an unknown entry. The variable '" \
                          + str(ln[0]) + "' is not an input variable for this model. Check " \
                          " the spelling of the variable name and only use the symbols : and = " \
                            "in variable assignments.\n"

        o.close()
            
        for k,v in self.input_file_vars.items():
            setattr(self, self.get_var_name(k), v)
            
        if verbose: print 'Finished reading ' + str(numvars) + ' variables from input file.'
            
    def set_defaults(self):
        
        for k,v in self._var_default_map.items(): setattr(self, self._var_name_map[k], v)
    
    def create_dicts(self):
                                       
        self._input_var_names = self._input_vars.keys()

        self._var_type_map = dict()
        self._var_name_map = dict()
        self._var_default_map = dict()

        for k in self._input_vars.keys():
            self._var_type_map[k] = self._input_vars[k]['type']
            self._var_name_map[k] = self._input_vars[k]['name']
            self._var_default_map[k] = self._input_vars[k]['default']

    def set_constants(self):

        self.g = 9.81   # (gravitation const.)
        
        
        self.dxn_iwalk = [1,1,0,-1,-1,-1,0,1]
        self.dxn_jwalk = [0,1,1,1,0,-1,-1,-1]
        self.dxn_dist = \
        [sqrt(self.dxn_iwalk[i]**2 + self.dxn_jwalk[i]**2) for i in range(8)]
        
        SQ05 = sqrt(0.5)
        self.dxn_ivec = [0,-SQ05,-1,-SQ05,0,SQ05,1,SQ05]
        self.dxn_jvec = [1,SQ05,0,-SQ05,-1,-SQ05,0,SQ05]

        self.walk_flat = np.array([1, -49, -50, -51, -1, 49, 50, 51])
        self.walk = np.array([[0,1], [-SQ05, SQ05], [-1,0], [-SQ05,-SQ05], 
                              [0,-1], [SQ05,-SQ05], [1,0], [SQ05,SQ05]])
            
    def create_other_variables(self):
        
        self.set_constants()
        
        self.theta_sand = self.coeff_theta_sand * self.theta_water
        self.theta_mud = self.coeff_theta_mud * self.theta_water
        
        self.U_dep_mud = self.coeff_U_dep_mud * self.u0
        self.U_ero_sand = self.coeff_U_ero_sand * self.u0
        self.U_ero_mud = self.coeff_U_ero_mud * self.u0
        
        self.L0 = int(round(self.L0_meters / self.dx))
        self.N0 = max(3,int(round(self.N0_meters / self.dx)))
        
        self.L = int(round(self.Length/self.dx))        # num cells in x
        self.W = int(round(self.Width/self.dx))         # num cells in y

        self.u_max = 2.0 * self.u0          # maximum allowed flow velocity
        
        self.C0 = self.C0_percent * 1/100                       # sediment concentration
  
        # (m) critial depth to switch to "dry" node
        self.dry_depth = min(0.1, 0.1*self.h0)
        self.CTR = floor(self.W/2)

        self.gamma = self.g * self.S0 * self.dx / (self.u0**2)

        self.V0 = self.h0 * (self.dx**2)    # (m^3) reference volume (volume to
                                            # fill cell to characteristic depth)

        self.Qw0 = self.u0 * self.h0 * self.N0 * self.dx    # const discharge
                                                            # at inlet        
                                                                                                       
        self.qw0 = self.u0 * self.h0                # water unit input discharge
        self.Qp_water = self.Qw0 / self.Np_water    # volume each water parcel

        self.qs0 = self.qw0 * self.C0               # sed unit discharge

        self.dVs = 0.1 * self.N0**2 * self.V0       # total amount of sed added 
                                                    # to domain per timestep

        self.Qs0 = self.Qw0 * self.C0           # sediment total input discharge
        self.Vp_sed = self.dVs / self.Np_sed    # volume of each sediment parcel
        
        self.itmax = 2 * (self.L + self.W)      # max number of jumps for parcel
        self.dt = self.dVs / self.Qs0           # time step size

        self.omega_flow_iter = 2 / self.itermax
     
        # number of times to repeat topo diffusion
        self.N_crossdiff = int(round(self.dVs / self.V0))
        
        # self.prefix
        self.prefix = self.out_dir
        if self.site_prefix:
            self.prefix += self.site_prefix + '_'
        if self.case_prefix:
            self.prefix += self.case_prefix + '_'
            
    
    def __init__(self):
        
        self.create_dicts()
        self.set_defaults()
        self.import_file()
        
        self.create_other_variables()



In [65]:
r = ReadInput()

Reading input file...
Finished reading 26 variables from input file.


NameError: global name 'sqrt' is not defined

In [24]:
# write parameters.yaml file
import os, sys, glob, string, shutil, re
import yaml, json
from collections import OrderedDict



class UnsortableList(list):
    def sort(self, *args, **kwargs):
        pass
    
class UnsortableOrderedDict(OrderedDict):
    def items(self, *args, **kwargs):
        return UnsortableList(OrderedDict.items(self, *args, **kwargs))
    
    
outFile = 'deltaRCM/db/parameters2.yaml'

input_vars = list()

entries = [['model_output__site_prefix', 'site_prefix', 'string', ''],
 ['model_output__case_prefix', 'case_prefix', 'string', ''],
 ['model_output__out_dir', 'out_dir', 'string', '~/deltaRCM_Output/'],
 ['model__total_timesteps', 'n_steps', 'long', 200],
 ['model_grid__length', 'Length', 'float', 200.0],
 ['model_grid__width', 'Width', 'float', 500.0],
 ['model_grid__cell_size', 'dx', 'float', 10.0],
 ['land_surface__width', 'L0_meters', 'float', 30.0],
 ['land_surface__slope', 'S0', 'float', 0.00015],
 ['model__max_iteration', 'itermax', 'long', 3],
 ['water__number_parcels', 'Np_water', 'long', 200],
 ['channel__flow_velocity', 'u0', 'float', 1.0],
 ['channel__width', 'N0_meters', 'float', 50.0],
 ['channel__flow_depth', 'h0', 'float', 5.0],
 ['sea_water_surface__elevation', 'H_SL', 'float', 0.0],
 ['sea_water_surface__rate_change_elevation', 'SLR', 'float', 0.0],
 ['sediment__number_parcels', 'Np_sed', 'long', 500],
 ['sediment__bedload_fraction', 'f_bedload', 'float', 0.25],
 ['sediment__influx_concentration', 'C0_percent', 'float', 0.1],
 ['model_output__opt_eta_figs', 'save_eta_figs', 'choice', False],
 ['model_output__opt_stage_figs', 'save_stage_figs', 'choice', False],
 ['model_output__opt_depth_figs', 'save_depth_figs', 'choice', False],
 ['model_output__opt_eta_grids', 'save_eta_grids', 'choice', False],
 ['model_output__opt_stage_grids', 'save_stage_grids', 'choice', False],
 ['model_output__opt_depth_grids', 'save_depth_grids', 'choice', False],
 ['model_output__opt_time_interval', 'save_dt', 'long', 10],
 ['coeff__surface_smoothing', 'Csmooth', 'float', 0.9],
 ['coeff__under_relaxation__water_surface', 'omega_sfc', 'float', 0.1],
 ['coeff__under_relaxation__water_flow', 'omega_flow', 'float', 0.9],
 ['coeff__iterations_smoothing_algorithm', 'Nsmooth', 'long', 5],
 ['coeff__depth_dependence__water', 'theta_water', 'float', 1.0],
 ['coeff__depth_dependence__sand', 'coeff_theta_sand', 'float', 2.0],
 ['coeff__depth_dependence__mud', 'coeff_theta_mud', 'float', 1.0],
 ['coeff__non_linear_exp_sed_flux_flow_velocity', 'beta', 'long', 3],
 ['coeff__sedimentation_lag', 'sed_lag', 'float', 1.0],
 ['coeff__velocity_deposition_mud', 'coeff_U_dep_mud', 'float', 0.3],
 ['coeff__velocity_erosion_mud', 'coeff_U_ero_mud', 'float', 1.5],
 ['coeff__velocity_erosion_sand', 'coeff_U_ero_sand', 'float', 1.05],
 ['coeff__topographic_diffusion', 'alpha', 'float', 0.1]]


In [25]:
yaml.add_representer(UnsortableOrderedDict, yaml.representer.SafeRepresenter.represent_dict)

for i in range(len(entries)):
    
    inner = UnsortableOrderedDict()
    value = UnsortableOrderedDict()
    
    inner['key'] = entries[i][0]
    inner['name'] = ''
    inner['description'] = ''
    
    value['type'] = entries[i][2]
    value['default'] = entries[i][3]
    value['units'] = ''
    
    inner['value'] = value
    
    input_vars.append(inner)
    
with open(outFile, 'w') as outfile:
    outStr = yaml.dump(input_vars, default_flow_style=False, stream = outfile)

In [26]:
input_vars

[OrderedDict([('key', 'model_output__site_prefix'),
              ('name', ''),
              ('description', ''),
              ('value',
               OrderedDict([('type', 'string'),
                            ('default', ''),
                            ('units', '')]))]),
 OrderedDict([('key', 'model_output__case_prefix'),
              ('name', ''),
              ('description', ''),
              ('value',
               OrderedDict([('type', 'string'),
                            ('default', ''),
                            ('units', '')]))]),
 OrderedDict([('key', 'model_output__out_dir'),
              ('name', ''),
              ('description', ''),
              ('value',
               OrderedDict([('type', 'string'),
                            ('default', '~/deltaRCM_Output/'),
                            ('units', '')]))]),
 OrderedDict([('key', 'model__total_timesteps'),
              ('name', ''),
              ('description', ''),
              ('value',
          